In [ ]:
"""

Purpose
-------
Generate publication-ready comparison figures for multiple prefilter configurations
(e.g., Pleated, Electrostatic, Hybrid, and optional Baseline) using wind-tunnel data.

This script produces four standalone figures:
  Fig 1) Gross ΔP normalized to a constant reference face velocity v_ref vs time
         + secondary x-axis in equivalent Martian sols
         + secondary y-axis with Mars-scaled ΔP at a target Mars velocity
  Fig 2) Prefilter ΔP (dp2) normalized to constant v_ref vs time
         + secondary x-axis in equivalent sols
         + secondary y-axis Mars-scaled ΔP
  Fig 3) Face velocity vs time + secondary x-axis in equivalent sols
  Fig 4) Gross ΔP normalized to constant v_ref vs dust loading
         (either areal loading kg/m² using FILTER_FACE_AREA_M2 or cumulative dust fed in g)

What this script does
---------------------
- Loads 3–4 CSV blocks (paste into raw_pleated/raw_electrostatic/raw_hybrid/(optional raw_baseline)).
- Cleans data: enforces required columns, converts to numeric, drops NaNs, sorts by time.
- Smooths dp1, dp2, and velocity using a centered rolling mean (optional).
- Computes:
    gross = dp1 + dp2
  and normalizes to constant velocity using a linear Darcy scaling:
    ΔP_const(t) = ΔP_meas(t) * (v_ref / v(t))
  (This assumes ΔP ∝ v, which is commonly appropriate in Darcy-flow regimes.)

- Adds optional Mars-context axes:
  * x-axis: seconds -> equivalent sols (via SOLS_PER_SEC)
  * y-axis: Earth ΔP -> Mars-scaled ΔP using a constant multiplier:
      ΔP_mars = ΔP_earth * (v_mars / v_ref) * slip_factor

CSV format (required columns)
-----------------------------
time,dp1,dp2,velocity

Units
-----
time: seconds
dp1, dp2: Pa
velocity: m/s

Outputs
-------
Saves PNG + PDF to ./fig_exports_compare
  fig1_compare_gross_const_time.(png|pdf)
  fig2_compare_prefilter_const_time.(png|pdf)
  fig3_compare_velocity_time.(png|pdf)
  fig4_compare_loading_vs_gross_const.(png|pdf)
"""

from __future__ import annotations

from io import StringIO
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


# =============================
# USER SETTINGS
# =============================
# Constant-velocity reference
V_REF = 0.75  # m/s

# Martian time conversion (example: 2200 s -> 68.7 sols)
SOL_REF_SECONDS = 2200.0
SOL_REF_SOLS = 68.7
SOLS_PER_SEC = SOL_REF_SOLS / SOL_REF_SECONDS

# Mars-context scaling (secondary y-axis)
V_MARS_TARGET = 0.071  # m/s (e.g., MOXIE-ish)
SLIP_FACTOR = 0.875    # dimensionless (e.g., 0.85–0.90 typical)

# Smoothing
ROLL_WIN = 9           # 1 disables smoothing; try 7–21 for smoother curves

# Robustness
MIN_VEL = 1e-6

# Dust loading inputs (Fig 4)
DUST_FEED_G_PER_S = 0.0091        # g/s
FILTER_FACE_AREA_M2 = 0.00543169  # m^2; set None to use cumulative grams instead

# Figure styling
LINE_WIDTH = 1.8
DPI = 300
OUTDIR = Path("fig_exports_compare")
OUTDIR.mkdir(parents=True, exist_ok=True)

AXIS_LABEL_SIZE = 14
TITLE_SIZE = 16
LEGEND_FONTSIZE = 10

YL_PRESSURE = "ΔP (Pa)"
YL_V_MPS = "Face velocity (m/s)"


# =============================
# COLORS (colorblind-friendly)
# =============================
C_PLEATED = "#0072B2"
C_ELEC    = "#E69F00"
C_HYBRID  = "#009E73"
C_BASE    = "#666666"  # baseline (optional)


# =============================
# DATA INPUTS (paste CSV blocks)
# =============================
raw_pleated = """
time,dp1,dp2,velocity
""".strip()

raw_electrostatic = """
time,dp1,dp2,velocity
""".strip()

raw_hybrid = """
time,dp1,dp2,velocity
""".strip()

# Optional baseline (no prefilter). Leave empty "" to disable.
raw_baseline = """
time,dp1,dp2,velocity
""".strip()


# =============================
# HELPERS
# =============================
REQUIRED_COLS = ["time", "dp1", "dp2", "velocity"]


def smooth(x: np.ndarray, win: int) -> np.ndarray:
    """Centered rolling-mean smoothing."""
    x = np.asarray(x, dtype=float)
    if not win or win < 2:
        return x
    # enforce odd window for symmetric smoothing
    if win % 2 == 0:
        win += 1
    return pd.Series(x).rolling(win, center=True, min_periods=1).mean().to_numpy()


def load_df(raw_csv: str) -> pd.DataFrame:
    """Parse a raw CSV string into a cleaned numeric DataFrame."""
    if not raw_csv.strip():
        raise ValueError("One of the datasets is empty. Paste CSV data into the raw_* string.")

    df = pd.read_csv(StringIO(raw_csv), comment="#")

    missing = [c for c in REQUIRED_COLS if c not in df.columns]
    if missing:
        raise ValueError(f"Missing required columns: {missing}. Required: {REQUIRED_COLS}")

    df = (
        df[REQUIRED_COLS]
        .apply(pd.to_numeric, errors="coerce")
        .dropna()
        .sort_values("time")
        .reset_index(drop=True)
    )

    if len(df) < 3:
        raise ValueError("Dataset has fewer than 3 valid rows after cleaning.")

    return df


def normalize_linear_darcy(dp: np.ndarray, v: np.ndarray, vref: float, min_vel: float = 1e-6) -> np.ndarray:
    """
    Linear Darcy constant-velocity reconstruction:
        ΔP_const(t) = ΔP_meas(t) * (vref / v(t))
    """
    v_use = np.maximum(np.asarray(v, dtype=float), min_vel)
    return np.asarray(dp, dtype=float) * (vref / v_use)


def add_secondary_sols_axis(ax: plt.Axes) -> plt.Axes:
    """Add a top x-axis converting seconds to equivalent Martian sols."""
    def sec_to_sols(x):  # seconds -> sols
        return x * SOLS_PER_SEC

    def sols_to_sec(x):  # sols -> seconds
        return x / SOLS_PER_SEC

    secax = ax.secondary_xaxis("top", functions=(sec_to_sols, sols_to_sec))
    secax.set_xlabel("Equivalent Martian sols", fontsize=AXIS_LABEL_SIZE)
    return secax


def add_mars_dp_secondary_yaxis(ax: plt.Axes, v_ref: float, v_mars: float, slip_factor: float) -> plt.Axes:
    """
    Add a right y-axis with a constant multiplier mapping:
      ΔP_mars = ΔP * (v_mars / v_ref) * slip_factor
    """
    scale = (v_mars / max(v_ref, 1e-12)) * slip_factor

    def earth_to_mars(dp):
        return np.asarray(dp) * scale

    def mars_to_earth(dp):
        return np.asarray(dp) / max(scale, 1e-12)

    secay = ax.secondary_yaxis("right", functions=(earth_to_mars, mars_to_earth))
    secay.set_ylabel(f"ΔP (Pa) — Mars-scaled @ {v_mars:.3f} m/s", fontsize=AXIS_LABEL_SIZE)
    return secay


def dust_loading_axis(time_s: np.ndarray, dust_feed_g_s: float, area_m2: float | None) -> tuple[np.ndarray, str]:
    """
    Compute x-axis dust loading based on constant dust feed rate.

    If area_m2 is None:
        x = cumulative dust fed (g)
    else:
        x = areal dust loading (kg/m²)
    """
    dust_rate_kg_s = max(float(dust_feed_g_s), 0.0) / 1000.0
    cum_mass_kg = dust_rate_kg_s * np.asarray(time_s, dtype=float)

    if area_m2 is None:
        x = cum_mass_kg * 1000.0
        x_label = "Cumulative dust fed (g)"
        return x, x_label

    area = float(area_m2)
    if area <= 0:
        raise ValueError("FILTER_FACE_AREA_M2 must be > 0, or set to None.")
    x = cum_mass_kg / area
    x_label = "Areal dust loading (kg/m²)"
    return x, x_label


def plot_common(ax: plt.Axes) -> None:
    """Common grid styling."""
    ax.grid(True, ls="--", alpha=0.35)


# =============================
# LOAD + PROCESS DATASETS
# =============================
datasets = [
    ("Pleated",       raw_pleated,       C_PLEATED),
    ("Electrostatic", raw_electrostatic, C_ELEC),
    ("Hybrid",        raw_hybrid,        C_HYBRID),
]

if raw_baseline.strip():
    datasets.append(("Baseline (no prefilter)", raw_baseline, C_BASE))

processed: list[dict[str, np.ndarray | str]] = []

for name, raw, color in datasets:
    df = load_df(raw)

    t = df["time"].to_numpy(dtype=float)

    v = smooth(df["velocity"].to_numpy(dtype=float), ROLL_WIN)
    v = np.maximum(v, MIN_VEL)

    dp1 = smooth(df["dp1"].to_numpy(dtype=float), ROLL_WIN)
    dp2 = smooth(df["dp2"].to_numpy(dtype=float), ROLL_WIN)
    gross = dp1 + dp2

    dp2_const = normalize_linear_darcy(dp2, v, V_REF, MIN_VEL)
    gross_const = normalize_linear_darcy(gross, v, V_REF, MIN_VEL)

    processed.append(
        {
            "name": name,
            "color": color,
            "t": t,
            "v": v,
            "dp2_const": dp2_const,
            "gross_const": gross_const,
        }
    )


# =============================
# FIG 1: Gross ΔP @ v_ref vs time + sols + Mars ΔP y-axis
# =============================
fig1, ax1 = plt.subplots(figsize=(6.8, 4.4))
for d in processed:
    ax1.plot(d["t"], d["gross_const"], lw=LINE_WIDTH, color=d["color"], label=d["name"])

ax1.set_xlabel("Experiment time (s)", fontsize=AXIS_LABEL_SIZE)
ax1.set_ylabel(f"Gross {YL_PRESSURE}", fontsize=AXIS_LABEL_SIZE)
ax1.set_title("Gross ΔP — Constant Face Velocity", fontsize=TITLE_SIZE)
plot_common(ax1)
ax1.legend(frameon=True, fontsize=LEGEND_FONTSIZE)

add_secondary_sols_axis(ax1)
add_mars_dp_secondary_yaxis(ax1, v_ref=V_REF, v_mars=V_MARS_TARGET, slip_factor=SLIP_FACTOR)

fig1.tight_layout()
fig1_png = OUTDIR / "fig1_compare_gross_const_time.png"
fig1_pdf = OUTDIR / "fig1_compare_gross_const_time.pdf"
fig1.savefig(fig1_png, dpi=DPI)
fig1.savefig(fig1_pdf)
plt.close(fig1)


# =============================
# FIG 2: Prefilter ΔP @ v_ref vs time + sols + Mars ΔP y-axis
# =============================
fig2, ax2 = plt.subplots(figsize=(6.8, 4.4))
for d in processed:
    ax2.plot(d["t"], d["dp2_const"], lw=LINE_WIDTH, color=d["color"], label=d["name"])

ax2.set_xlabel("Experiment time (s)", fontsize=AXIS_LABEL_SIZE)
ax2.set_ylabel(f"Prefilter {YL_PRESSURE}", fontsize=AXIS_LABEL_SIZE)
ax2.set_title("Prefilter ΔP — Constant Face Velocity", fontsize=TITLE_SIZE)
plot_common(ax2)
ax2.legend(frameon=True, fontsize=LEGEND_FONTSIZE)

add_secondary_sols_axis(ax2)
add_mars_dp_secondary_yaxis(ax2, v_ref=V_REF, v_mars=V_MARS_TARGET, slip_factor=SLIP_FACTOR)

fig2.tight_layout()
fig2_png = OUTDIR / "fig2_compare_prefilter_const_time.png"
fig2_pdf = OUTDIR / "fig2_compare_prefilter_const_time.pdf"
fig2.savefig(fig2_png, dpi=DPI)
fig2.savefig(fig2_pdf)
plt.close(fig2)


# =============================
# FIG 3: Face velocity vs time + sols
# =============================
fig3, ax3 = plt.subplots(figsize=(6.8, 4.4))
for d in processed:
    ax3.plot(d["t"], d["v"], lw=LINE_WIDTH, color=d["color"], label=d["name"])

ax3.set_xlabel("Experiment time (s)", fontsize=AXIS_LABEL_SIZE)
ax3.set_ylabel(YL_V_MPS, fontsize=AXIS_LABEL_SIZE)
ax3.set_title("Face Velocity vs Time", fontsize=TITLE_SIZE)
plot_common(ax3)
ax3.legend(frameon=True, fontsize=LEGEND_FONTSIZE)

add_secondary_sols_axis(ax3)

fig3.tight_layout()
fig3_png = OUTDIR / "fig3_compare_velocity_time.png"
fig3_pdf = OUTDIR / "fig3_compare_velocity_time.pdf"
fig3.savefig(fig3_png, dpi=DPI)
fig3.savefig(fig3_pdf)
plt.close(fig3)


# =============================
# FIG 4: Dust loading vs Gross ΔP @ constant v_ref
# =============================
fig4, ax4 = plt.subplots(figsize=(6.6, 4.4))

x_label = "Dust loading"
for d in processed:
    x, x_label = dust_loading_axis(d["t"], DUST_FEED_G_PER_S, FILTER_FACE_AREA_M2)
    ax4.plot(x, d["gross_const"], lw=LINE_WIDTH, color=d["color"], label=d["name"])

ax4.set_xlabel(x_label, fontsize=AXIS_LABEL_SIZE)
ax4.set_ylabel(YL_PRESSURE, fontsize=AXIS_LABEL_SIZE)
ax4.set_title("Gross ΔP vs Dust Loading — Constant Face Velocity", fontsize=TITLE_SIZE)
plot_common(ax4)
ax4.legend(frameon=True, fontsize=LEGEND_FONTSIZE)

fig4.tight_layout()
fig4_png = OUTDIR / "fig4_compare_loading_vs_gross_const.png"
fig4_pdf = OUTDIR / "fig4_compare_loading_vs_gross_const.pdf"
fig4.savefig(fig4_png, dpi=DPI)
fig4.savefig(fig4_pdf)
plt.close(fig4)


print("Saved:")
print(f"  {fig1_png} | {fig1_pdf}")
print(f"  {fig2_png} | {fig2_pdf}")
print(f"  {fig3_png} | {fig3_pdf}")
print(f"  {fig4_png} | {fig4_pdf}")

Saved:
  fig_exports_compare/fig1_compare_gross_const_time.png | fig_exports_compare/fig1_compare_gross_const_time.pdf
  fig_exports_compare/fig2_compare_prefilter_const_time.png | fig_exports_compare/fig2_compare_prefilter_const_time.pdf
  fig_exports_compare/fig3_compare_velocity_time.png | fig_exports_compare/fig3_compare_velocity_time.pdf
  fig_exports_compare/fig4_compare_loading_vs_gross_const.png | fig_exports_compare/fig4_compare_loading_vs_gross_const.pdf
